In [9]:
import pandas as pd
import os
from collections import defaultdict
import warnings

# Disable PyArrow string type to avoid extension type conflicts
pd.options.mode.string_storage = "python"

# Suppress ArrowKeyError warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [8]:
race_results_path = "/Users/thatcher/dev/analysis/projects/marathon_results/race_results"

total_races = len(os.listdir(race_results_path))
print(f"Total Races: {total_races}")

column_concordance = defaultdict(int)
for race_result in os.listdir(race_results_path):
    race_file = os.path.join(race_results_path, race_result)
    #print(race_file)
    df = pd.read_parquet(race_file)
    for c in df.columns:
        column_concordance[c] += 1

for column, count in column_concordance.items():
    print(f"{column} - {count} - {count*1.0/total_races}")

Total Races: 1836


ArrowKeyError: A type extension with name pandas.period already defined

In [5]:
primary_name_field = "Last Name, First Name (Sex/Age)"
required_columns = [primary_name_field, "Time", "date", "BQ*", "City, State, Country"]
def has_enough_information(frame):
    columns = set(frame.columns)
    for required_column in required_columns:
        if required_column not in columns:
            return False
    return True

total_races = 0
total_race_days = 0
total_racers = 0
races_with_enough_info = []
for race_result in os.listdir(race_results_path):
    race_file = os.path.join(race_results_path, race_result)
    df = pd.read_parquet(race_file)
    if has_enough_information(df):
        total_races += 1
        total_racers += df.shape[0]
        total_race_days += len(df["date"].unique())
        races_with_enough_info.append(race_result)

print(f"Total Races: {total_races}")
print(f"Total Racers: {total_racers}")
print(f"Total Race Days {total_race_days}")

Total Races: 1094
Total Racers: 10358373
Total Race Days 10964


In [11]:
import heapq

ranked_races = []
for race in races_with_enough_info:
    race_file = os.path.join(race_results_path, race)
    df = pd.read_parquet(race_file)
    heapq.heappush(ranked_races, (df.shape[0], race))
top_races = heapq.nlargest(20, ranked_races)

total_racers_in_selection = sum([x[0] for x in top_races])

print(top_races)
print(total_racers_in_selection*1.0 / total_racers)

for (races, race) in top_races:
    print(race)

[(903054, 'new_york_city_marathon.parquet'), (797446, 'chicago_marathon.parquet'), (454282, 'boston_marathon.parquet'), (447105, 'honolulu_marathon.parquet'), (414742, 'marine_corps_marathon.parquet'), (401741, 'city_of_los_angeles_marathon_(l.a._marathon).parquet'), (290029, 'disney_world_marathon.parquet'), (219509, "rock_'n'_roll_marathon.parquet"), (183190, 'philadelphia_marathon.parquet'), (165283, 'twin_cities_marathon.parquet'), (145597, "grandma's_marathon.parquet"), (141268, 'houston_marathon.parquet'), (120492, 'portland_marathon.parquet'), (111870, 'california_international_marathon.parquet'), (110500, 'st._george_marathon.parquet'), (104218, 'the_san_francisco_marathon.parquet'), (87300, 'melbourne_marathon.parquet'), (87083, 'cincinnati_flying_pig_marathon.parquet'), (85498, 'columbus_marathon.parquet'), (85381, 'richmond_marathon.parquet')]
0.5170298462895669
new_york_city_marathon.parquet
chicago_marathon.parquet
boston_marathon.parquet
honolulu_marathon.parquet
marine_c

In [ ]:
location_map = pd.read_csv("location_mapping.csv",names=["race","race_city","race_state","race_country"])
all_dataframes = []
for race in location_map.race:
    race_file = os.path.join(race_results_path,race+".parquet")
    all_dataframes.append(pd.read_parquet(race_file))
combined_frame = pd.concat(all_dataframes)
combined_frame

FileNotFoundError: [Errno 2] No such file or directory: '/home/thatcher/dev/marathon/race_results/new_york_city_marathon'

National Climate Data Center Archive: https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/